# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [3]:
#     def d(x):
#         if 'City' not in str(x) and 'Area' not in str(x):
#             return x
#         else:
#             if 'City' in str(x):
#                 ind = str(x).find('City')
#                 x = str(x)[:(ind-1)]
#                 return x
#             else:
#                 ind = str(x).find('Area')
#                 x = str(x)[:(ind-1)]
#                 return x
            
#     def do(x):
#         if '*' in str(x):
#             return x[:-1]
#         else:
#             return x
        
#     # print(df.sort_values(by=['team'])['team'])
    
#     def do_func(x):
#         if str(x) in ' '.join(list(df['team'])):
#             for i in list(df['team']):
#                 if str(x) in i:
#                     print(i)
#                     return float(df.set_index('team').loc[str(i.strip()), 'W']) / float(df.set_index('team').loc[str(i.strip()), 'L'])
    
#     new_df = cities[['Metropolitan area', 'Population']]
#     df['team'] = df['team'].apply(do)
#     new_df['Metropolitan area'] = new_df['Metropolitan area'].apply(d)
#     new_df['Ratio'] = new_df['Metropolitan area'].apply(do_func)
    

import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    df = nhl_df[1:35]
    df = df.drop([9, 18, 26])
    
    df = df.sort_values(by=['team'])
    
    # cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.iloc[:-1,[0,3,8]]
    cities = cities.rename(columns={'Population (2016 est.)[8]' : 'Population'})
    # cities = cities.sort_values(by='Metropolitan area', axis=0)
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = []
    
    def check_func(x):
        if '—' in str(x):
            return np.NaN
        elif '[' in str(x):
            x = x.split('[')[0]
            return x
        else:
            return x
    
    def del_empty(x):
        if not x:
            return np.NaN
        else:
            return x
    
    def make_ratio(x):
        for i in list(cities['NHL']):
            if str(i.strip()) in str(x.loc['team']):
                return float(x.loc['Ratio'])
            
    cities['NHL'] = cities['NHL'].apply(check_func)
    cities['NHL'] = cities['NHL'].apply(del_empty)
    
    cities = cities.dropna()
    
    df['Ratio'] = df[['W', 'L']].apply(lambda x: float(x.loc['W']) / float(x.loc['L']), axis=1)
    
    
    cities['Ratio'] = df[['team', 'Ratio']].apply(make_ratio, axis=1)
    print(df)
    print(cities)
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
nhl_correlation()

                      team  GP   W   L  OL  PTS  PTS%   GF   GA    SRS    SOS  \
28          Anaheim Ducks*  82  44  25  13  101  .616  235  216   0.24   0.01   
34         Arizona Coyotes  82  29  41  12   70  .427  208  256  -0.53   0.05   
2           Boston Bruins*  82  50  20  12  112  .683  270  214   0.62  -0.07   
8           Buffalo Sabres  82  25  45  12   62  .378  199  280  -0.98   0.01   
31          Calgary Flames  82  37  35  10   84  .512  218  248  -0.33   0.03   
15     Carolina Hurricanes  82  36  35  11   83  .506  228  256  -0.35  -0.01   
25      Chicago Blackhawks  82  33  39  10   76  .463  229  256  -0.26   0.07   
22     Colorado Avalanche*  82  43  30   9   95  .579  257  237   0.28   0.04   
13  Columbus Blue Jackets*  82  45  30   7   97  .591  242  230   0.11  -0.04   
24            Dallas Stars  82  42  32   8   92  .561  235  225   0.17   0.04   
5        Detroit Red Wings  82  30  39  13   73  .445  217  255  -0.48  -0.01   
32         Edmonton Oilers  

AssertionError: Q1: There should be 28 teams being analysed for NHL

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values